# Inverse problem of the reconstruction of the Neumann BCs for Laplace's equation

__Formulation of Inverse Problem__

__Direct Problem__
- Equation: $A * x = f$
- $A$: Matrix of Direct Problem
  - shape: ($\text{dof}$, $\text{dof}$)
- $x$: Solution of Direct Problem
  - shape: ($\text{dof}$)
- $f$: Load Vector of Direct Problem
  - shape: ($\text{dof}$)
- $F$: Projection Matrix from Normal Derivative in given points (Mass Matrix over Boundary Elements)
  - shape: ($\text{dof}$, $g$)

__Inverse Problem__
- Equation: $M * y = \text{grad}$
- $M$: Resulting Matrix of Inverse Problem
  - shape: ($3p$, $\text{dof}$)
- $y$: Normal Derivative in Boundary Points
- $\text{grad}$: Concatenation of Grad Vectors ($\text{grad}_x, \text{grad}_y, \text{grad}_z$)

__Additional Variables__
- $P_x, P_y, P_z$: Projection Matrices from Solution to Partial Derivative
  - shape: ($p$, $\text{dof}$)
- $M_x = P_x \times A^{-1} \times F$: Need batching with Direct Solvers (spsolve or better factorized to reduce amount of time)
  - Note: $A$ is singular, so we need to be careful with solving the system
- $M = (M_x, M_y, M_z)$: Concatenation of Inverse Problem Matrices
- $\text{dof}$: Degrees of Freedom of Direct Problem
- $p$: Number of Measuring Points
- $g$: Number of Boundary Points with Known Normal Derivative (Is it all boundary points?)

__About calculating M__

Computing $M$ consists of solving a lot of systems of linear equations:

$$A^{-1} * F = X$$

is same with solving SLAE with matrix rhs:

$$AX = F$$

Result of this multipication can be a big dense (90% density) matrix so we should solve SLAE with one or several columns form $F$.

Probable algorithm for getting $M$ (Under investigation, because it gives big error (e.g. `1e-2` instead of `1e-11`))

```python
for f in F:

  x = spsolve(A, f) # f.shape: (dof, 1)
  
  Mx[i] = P_x @ x
```

__Questions__

1. How to deal with $A$ is singular? It affects calculation of $M$.

2. How to assemble $F$, if we have rhs (source) in direct problem? (If equation is not Laplace's)

## Preprocessing

Import the required libraries

In [25]:
import numpy as np
import scipy as sp

from tqdm import tqdm

import pyquasar as pq

np.set_printoptions(precision=3, suppress=False)

Load the `.geo` file and generate the mesh for the problem using GMSH.

`refine_k` is the number of times the mesh is refined.

In [56]:
mesh = pq.Mesh.load("cylinder.geo", refine_k=2)
mesh

<Mesh object summary 
	Numeration: global
	Domains: [<MeshDomain object summary
	Material: Air
	Total elements number: 55808
	Element type: Tetrahedron 4; Count: 55808
	Boundary type: neumann; Tag: 1; Element type: Triangle 3; Count: 4960.
	Boundary type: neumann; Tag: 2; Element type: Triangle 3; Count: 1024.
	Boundary type: neumann; Tag: -3; Element type: Triangle 3; Count: 1024.
>]>

## Direct Problem

Assemble & solve the direct problem to obtain problem matrix, load vector and the solution `u`.

Note that mesh consists of only Neumann boundary conditions and therefore the solution `u` is need to be orthogonalized.

In [27]:
# Define the materials dictionary
materials = {
  "air": {"air": 0},
}

# Create a list of FemDomain objects from the mesh domains
domains = [pq.FemDomain(domain) for domain in mesh.domains]

# Create a FemProblem object with the domains
problem = pq.FemProblem(domains)

# Assemble the problem using the materials dictionary
problem.assembly(materials)

# Print the degree of freedom count
print(f"DOF: {problem.dof_count}")

problem._matrix = problem._matrix.tocsr()
problem.matrix.data[problem.matrix.indptr[-2] : problem.matrix.indptr[-1]] = 0
problem._matrix = problem._matrix.tocsc()
problem.matrix.data[problem.matrix.indptr[-2] : problem.matrix.indptr[-1]] = 0
problem._matrix[-1, -1] = 1
problem._matrix.eliminate_zeros()

DOF: 11107


## Inverse Problem

Get points of interest

In [61]:
data = np.load("data/data.npz")
xs = data["xs"]
ys = data["ys"]
zs = data["zs"]
grad_x = data["grad_x"]
grad_y = data["grad_y"]
grad_z = data["grad_z"]
data.close()

In [62]:
pts = np.concatenate([xs[:7200, None], ys[:7200, None], zs[:7200, None]], axis=1)
pts.shape

(7200, 3)

Project the solution gradient into the points of interest

In [30]:
proj_grad = problem.project_grad_into(pts, batch_size=128)

100%|██████████| 57/57 [00:29<00:00,  1.94it/s]


In [31]:
grad = np.concatenate([grad_x[:7200], grad_y[:7200], grad_z[:7200]])

Get the Mass matrix over the Neumann type boundary

In [32]:
# POSSIBLE CHECK: F[:boundary_ids.max(),]^-1 @ load -> best approximation of the flow
F = problem.mass_boundary(["neumann"])

Initialize the inverse problem matrix

In [33]:
m_shape = pts.shape[0], F.shape[1]
Mx = np.zeros(m_shape)
My = np.zeros(m_shape)
Mz = np.zeros(m_shape)

Get factorization of direct problem matrix

In [34]:
sp.sparse.linalg.use_solver(useUmfpack = False)

In [35]:
direct_problem_solve = sp.sparse.linalg.factorized(problem.matrix)

Iteratively calculate the inverse problem matrix

In [36]:
for i, f in tqdm(enumerate(F.T), total=F.shape[1]):
  sol = direct_problem_solve(f.T.toarray().ravel())

  Mx[:, i] = (proj_grad[0] @ sol).ravel()
  My[:, i] = (proj_grad[1] @ sol).ravel()
  Mz[:, i] = (proj_grad[2] @ sol).ravel()

M = np.concatenate([Mx, My, Mz])

100%|██████████| 3506/3506 [00:51<00:00, 67.92it/s]


Iteratively calculate the inverse problem solution

In [44]:
res_flow, istop, itn, normr = sp.sparse.linalg.lsmr(M, grad, atol=1e-15, btol=1e-15, maxiter=10000)[:4]

rhs = F @ res_flow
rerr = np.linalg.norm(M @ res_flow - grad) / np.linalg.norm(grad)

print(f"The reason of stopping: {istop}")
print(f"Number of iterations: {itn}")
print(f"Norm of the residual: {normr:.2e}")
print(f"Relative error of inverse problem solution (M @ y = grad): {rerr:.2e}")

The reason of stopping: 7
Number of iterations: 10000
Norm of the residual: 5.07e+00
Relative error of inverse problem solution (M @ y = grad): 2.73e-02


Test through direct problem

In [45]:
mat_dict = {
  "air": {"air": 0},
}

test_problem = pq.FemProblem(domains)

# Assemble the problem using the materials dictionary
test_problem.assembly(mat_dict)

# Print the degree of freedom count
print(f"DOF: {test_problem.dof_count}")

test_problem._matrix = test_problem._matrix.tocsr()
test_problem.matrix.data[test_problem.matrix.indptr[-2] : test_problem.matrix.indptr[-1]] = 0
test_problem._matrix = test_problem._matrix.tocsc()
test_problem.matrix.data[test_problem.matrix.indptr[-2] : test_problem.matrix.indptr[-1]] = 0
test_problem._matrix[-1, -1] = 1
test_problem._matrix.eliminate_zeros()

# Get the kernel for Gram-Schmidt orthogonalization
kernel = test_problem.domains[0].kernel

test_problem._load_vector += F @ res_flow

test_problem.factorize()
# Solve the problem
test_sol = test_problem.solve(atol=1e-15)

# Perform the Gram-Schmidt orthogonalization
test_sol -= kernel[0] * (test_sol @ kernel[0]) / (kernel[0] @ kernel[0])

DOF: 11107


In [46]:
test_pts = np.concatenate([xs[7200:, None], ys[7200:, None], zs[7200:, None]], axis=1)

In [47]:
test_true_grad = np.concatenate([grad_x[7200:], grad_y[7200:], grad_z[7200:]])

In [48]:
test_proj_grad = test_problem.project_grad_into(test_pts, batch_size=128)
test_grad_x = test_proj_grad[0] @ test_sol
test_grad_y = test_proj_grad[1] @ test_sol
test_grad_z = test_proj_grad[2] @ test_sol

test_grad = np.concatenate([test_grad_x, test_grad_y, test_grad_z])

100%|██████████| 1145/1145 [08:35<00:00,  2.22it/s]


In [49]:
np.linalg.norm(test_grad - test_true_grad) / np.linalg.norm(test_true_grad)

1.4910377334221272

In [50]:
test_grad - test_true_grad

array([ 3.857e-02, -1.363e-02, -5.812e-04, ..., -1.135e+01, -6.319e+00,
       -2.405e+01])